# Dates Bar Charts - Fake Covid-19 Dataset

### fake_dates_bar_chart.py and fake_dates_area_chart.py

In order to create the bar and area charts, we need the following packages:

In [1]:
from collections import Counter
import pandas as pd
from dateutil.parser import parse
import datetime
#import numpy as np
import json
#import sys
#import string
#import re
import itertools  
#from emot.emo_unicode import UNICODE_EMO, EMOTICONS
#import emoji
#from wordcloud import WordCloud, STOPWORDS
#import matplotlib.pyplot as plt 
import nltk
#from PIL import Image
import altair as alt
import csv
#import itertools

To read the JSON file that has all the tweets, it is necessary to do:

In [16]:
data = []
with open('dataset/fakecovid_result_final_translated_full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

### Multi Line Chart: number of tweets created each month and classified by their category

Here the goal is still to create a multi line chart that shows the trend of the number of tweets created during the year (grouped by month) but here we are going to visualize two lines: one for the FALSE tweets and one for the PARTIALLY FALSE tweets:

In [17]:
csv_dataframe = pd.read_csv('dataset/FINAL_fakecovid_final_filtered_dataset_clean.csv',sep=";")
csv_dataframe['tweet_id'] = csv_dataframe['tweet_id'].astype(str)
csv_list = csv_dataframe.values.tolist()
lista_unica_csv=list(itertools.chain.from_iterable(csv_list))


index=0
cmt_list = [] 
for element in data:
    token_id = data[index]['id_str']
    indice_csv = lista_unica_csv.index(token_id)
    token = data[index]['created_at']
    d = parse(token)
    d = d.strftime('%Y/%m/%d')
    final_token = d + " " +lista_unica_csv[indice_csv+1].lower().replace(" ", "")
    cmt_list.append(final_token)
    index=index+1
    
count=Counter(cmt_list)

#start = datetime.datetime.strptime("2020/01/01", "%Y/%m/%d")
#end = datetime.datetime.strptime("2020/09/01", "%Y/%m/%d")
#date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

#for date in date_generated:
#    if date.strftime("%Y/%m/%d") not in count:
#        count[date.strftime("%Y/%m/%d") + " " + "nulla"] = 0
        
df = pd.DataFrame.from_dict(count, orient='index').reset_index()
df = df.rename(columns={'index':'data', 0:'count'})
#print(df.to_string())

Once the dataset is created, we work on it to create a better visualisation:

In [18]:
col_one_list = df['data'].tolist()
col_two_list = df['count'].tolist()

typelist=[]
namelist=[]

index = 0

count_false = [0] * len(col_one_list)
count_part = [0] * len(col_two_list)

for el in col_one_list:
    tok = el.split()
    namelist.append(tok[0])
    #typelist.append(tok[1])
    if tok[0] in namelist:
        print(tok)
        indx = namelist.index(tok[0])
        if tok[1] == "false":
            count_false[indx] = col_two_list[index]
        elif tok[1] == "partiallyfalse":
            count_part[indx] = col_two_list[index]
        else:
            print("nulla")
   
    index = index + 1

i=0
for el in col_two_list:
    col_two_list[i] = count_false[i] + count_part[i]
    i = i + 1

df['data']=namelist
df['Count False Tweets']=count_false
df['Count Partially False Tweets']=count_part
df['count'] = col_two_list

df = df.sort_values(by=['data'])
print(df.head(30))

['2020/02/02', 'false']
['2020/02/11', 'false']
['2020/02/15', 'false']
['2020/02/16', 'false']
['2020/03/04', 'partiallyfalse']
['2020/03/11', 'false']
['2020/03/24', 'false']
['2020/03/26', 'false']
['2020/03/27', 'partiallyfalse']
['2020/04/08', 'false']
['2020/04/09', 'false']
['2020/04/15', 'false']
['2020/04/21', 'false']
['2020/04/24', 'false']
['2020/04/28', 'false']
['2020/05/03', 'false']
['2020/05/06', 'partiallyfalse']
['2020/05/07', 'false']
['2020/05/10', 'false']
['2020/05/16', 'false']
['2020/05/19', 'partiallyfalse']
['2020/05/20', 'false']
['2020/05/21', 'false']
['2020/05/24', 'false']
['2020/05/25', 'false']
['2020/05/26', 'partiallyfalse']
['2020/05/26', 'false']
['2020/05/27', 'partiallyfalse']
['2020/05/27', 'false']
['2020/06/01', 'partiallyfalse']
['2020/06/04', 'false']
['2020/06/06', 'false']
['2020/06/08', 'false']
['2020/06/08', 'partiallyfalse']
['2020/06/09', 'false']
['2020/06/10', 'false']
['2020/06/11', 'false']
['2020/06/11', 'partiallyfalse']
['2020/

Plot the chart and add the tooltip for a better experience:

In [37]:

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['data'], empty='none')



lines = alt.Chart(df).transform_fold(
    ['Count False Tweets', 'Count Partially False Tweets'],
).mark_line().encode(
    x=alt.X('data:T',axis=alt.Axis(format='%B %d %Y')),
    y=alt.Y('value:Q'),
    color=alt.Color('key:N'),
    tooltip=[
        alt.Tooltip('monthdate(data):T', title='Date'),
        alt.Tooltip('value:Q', title='Tweet Count')
    ]
).transform_filter(
    alt.FieldRangePredicate(field='count', range=[1, 1000])
).properties(title="aaa")

selectors = alt.Chart(df).mark_point().encode(
    x=alt.X('data:T',axis=alt.Axis(format='%B %d %Y')),
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = lines.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)


# Draw text labels near the points, and highlight based on selection
text = lines.transform_fold(
    ['Count False Tweets', 'Count Partially False Tweets'],
).mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'value:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(df).mark_rule(color='gray').encode(
    x=alt.X('data:T',axis=alt.Axis(format='%B %d %Y')),
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    lines, selectors, points, rules, text
).properties(
    width=1000, height=500
)

alt.LayerChart(...)

In [46]:
brush = alt.selection(type='interval')

points = alt.Chart(df).transform_fold(
    ['Count False Tweets', 'Count Partially False Tweets'],
).mark_point().encode(
    x=alt.X('month(data):T'),
    y='value:Q',
    color=alt.condition(brush, 'key:N', alt.value('lightgray'))
).transform_filter(
    alt.FieldRangePredicate(field='count', range=[10, 1000])
).add_selection(
    brush
)

bars = alt.Chart(df).transform_fold(
    ['Count False Tweets', 'Count Partially False Tweets'],
).mark_bar().encode(
    y='value:Q',
    color='key:N',
    x='key:N'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)